# Sizing Modes
## Testing the different sizing modes for converter components

For the converters involved in this example, three different parameters have been defined, which determine how the `PerformanceModel.compute()` function will resize the capacity.

- `normal`: In this mode, converters function as they always have previously:
    - The size of the asset is fixed within the `compute()` function, and is read from the config.
- `resize_by_max_feedstock`: In this mode, the size of the asset is adjusted to be able to utilize all of the available feedstock:
    - The size of the asset is *calculated* within the `compute()` function, by how big it needs to be to utilize all of the `<feedstock>_in`.
- `resize_by_max_commodity`: In this mode, the size of the asset is adjusted to be able to supply its product to the full capacity of another downstream converter:
    - The size of the asset is calculated within the `compute()` function, by how big it needs to be to match the capacity of a connected downstream tech.
    - The asset will use a connected variable `max_<commodity>_capacity` from the downstream tech. This creates a feedback loop.
     

## Example plant setup

Here, there are four technologies in the the `tech_config.yaml`: An `electrolyzer`, a `hopp` (configured for electricity), `h2_storage` and `ammonia` (to consume the hydrogen).

### Example technology_interconnections within plant_config.yaml

The final connection from the ammonia back to the electroyzer is used to size the electrolyzer.

```
technology_interconnections:
- ["hopp", "electrolyzer", "electricity"]
- ["electrolyzer", "h2_storage", "hydrogen"]
- ["h2_storage", "ammonia", "hydrogen"]
- ["ammonia", "electrolyzer", "max_hydrogen_capacity"]
```

### Example tech_config for the `electrolyzer` within tech_config_size_mode_commodity_iterative.yaml

The tech_config for the electrolyzer is configured to be sized by the `resize_by_max_commodity` mode.
The sizing flow is `hydrogen`, and it will be sized to accommodate the feedstock demand of the connected technology, `ammonia`.

```
electrolyzer:
    model_inputs:
      shared_parameters:
        n_clusters: 1
        location: "onshore"
        cluster_rating_MW: 5.0
        pem_control_type: "basic"
        eol_eff_percent_loss: 10
      performance_parameters:
        sizing: 
          size_mode: "resize_by_max_commodity"
          size_flow: "hydrogen"
          size_tech: "ammonia"
```

### Example tech_config for the `ammonia` within tech_config_size_mode_commodity_iterative.yaml

For the ammonia tech_config, the sizing mode is set to `normal`, so it will be sized at the capacity value in the config.

```
ammonia:
  ...
  performance_parameters:
    sizing:
      size_mode: "normal"
    capacity_factor: 0.9
    ...
```

## Running an example

### `normal` mode

The example `.yaml` files have been set up in `normal` mode.
In this case the electrolyzer has been sized to 640 MW (same as previous example), but the electricity profile going in has a max of over 1000 MW.
The LCOH is $4.64/kg H2 and the LCOA is $1.14/kg NH3.

In [1]:
from h2integrate.core.h2integrate_model import H2IntegrateModel


# Create a H2Integrate model
model = H2IntegrateModel("14_size_mode_normal.yaml")

# Run the model
model.run()

model.post_process()

XDSM diagram written to connections_xdsm.pdf


77 explicit inputs:

╷                  ╷           ╷       ╷                                      
 Variable                            │             Mean │ Units     │ Shape │ Promoted name                        
╶────────────────────────────────────┼──────────────────┼───────────┼───────┼─────────────────────────────────────╴
 plant                               │                  │           │       │                                      
   ammonia                           │                  │           │       │                                      
     synloop_ammonia_cost            │                  │           │       │                                      
       total_ammonia_produced        │   396,940,725.31 │ kg/year   │ 1     │ ammonia.total_ammonia_produced       
       total_hydrogen_consumed       │    72,043,515.94 │ kg/year   │ 1     │ ammonia.total_hydrogen_consumed      
       total_nitrogen_consumed       │ 3,003,314,070.67 │ kg/year   │ 1     │ ammonia.total_nitrogen_consumed      
       total_electricity_consumed    │   245,334,157.62 │ kW*h/year │ 1     │ ammonia.total_electricity_consumed   
     synloop_ammonia_performance     │                  │           │       │                                      
       hydrogen_in                   │       8,224.1457 │ kg/h      │ 8760  │ ammonia.hydrogen_in                  
       nitrogen_in                   │           0.0000 │ kg/h      │ 8760  │ ammonia.nitrogen_in                  
       electricity_in                │           0.0000 │ MW        │ 8760  │ ammonia.electricity_in               
   electrolyzer                      │                  │           │       │                                      
     eco_pem_electrolyzer_performanc │                  │           │       │                                      
 e                                   │                  │           │       │                                      
       electricity_in                │       465,297.78 │ kW        │ 8760  │ electrolyzer.electricity_in          
     singlitico_electrolyzer_cost    │                  │           │       │                                      
       electricity_in                │       465,297.78 │ kW        │ 8760  │ electrolyzer.electricity_in          
       n_clusters                    │          16.0000 │ unitless  │ 1     │ electrolyzer.n_clusters              
       max_hydrogen_capacity         │       1,000.0000 │ kg/h      │ 1     │ electrolyzer.max_hydrogen_capacity   
       total_hydrogen_produced       │    69,067,039.80 │ kg/year   │ 1     │ electrolyzer.total_hydrogen_produced 
       electrolyzer_size_mw_cost     │         640.0000 │ MW        │ 1     │ electrolyzer.electrolyzer_size_mw_co 
                                     │                  │           │       │ st                                   
   electrolyzer_to_h2_storage_pipe   │                  │           │       │                                      
     hydrogen_in                     │           2.2845 │ kg/s      │ 8760  │ electrolyzer_to_h2_storage_pipe.hydr 
                                     │                  │           │       │ ogen_in                              
   finance_subgroup_h2               │                  │           │       │                                      
     adjusted_capex_opex_comp        │                  │           │       │                                      
       capex_hopp                    │ 1,754,699,616.00 │ USD       │ 1     │ finance_subgroup_h2.capex_hopp       
       opex_hopp                     │    32,953,490.40 │ USD/year  │ 1     │ finance_subgroup_h2.opex_hopp        
       varopex_hopp                  │           0.0000 │ USD/year  │ 30    │ finance_subgroup_h2.varopex_hopp     
       capex_electrolyzer            │   600,412,523.71 │ USD       │ 1     │ finance_subgroup_h2.capex_electrolyz 
                                     │                  │           │       │ er             

107 explicit outputs:

╷                  ╷           ╷       ╷                                      
 Variable                            │             Mean │ Units     │ Shape │ Promoted name                        
╶────────────────────────────────────┼──────────────────┼───────────┼───────┼─────────────────────────────────────╴
 plant                               │                  │           │       │                                      
   ammonia                           │                  │           │       │                                      
     synloop_ammonia_cost            │                  │           │       │                                      
       CapEx                         │ 1,151,737,526.88 │ USD       │ 1     │ ammonia.CapEx                        
       OpEx                          │    25,486,491.18 │ USD/year  │ 1     │ ammonia.OpEx                         
       VarOpEx                       │           0.0000 │ USD/year  │ 30    │ ammonia.VarOpEx                      
       capex_asu                     │   347,075,632.70 │ USD       │ 1     │ ammonia.capex_asu                    
       capex_synloop                 │   443,088,488.84 │ USD       │ 1     │ ammonia.capex_synloop                
       capex_boiler                  │    10,355,840.22 │ USD       │ 1     │ ammonia.capex_boiler                 
       capex_cooling_tower           │     7,030,562.36 │ USD       │ 1     │ ammonia.capex_cooling_tower          
       capex_direct                  │   807,550,524.13 │ USD       │ 1     │ ammonia.capex_direct                 
       capex_depreciable_nonequipmen │   344,187,002.75 │ USD       │ 1     │ ammonia.capex_depreciable_nonequipme 
 t                                   │                  │           │       │ nt                                   
       land_cost                     │     5,015,782.62 │ USD       │ 1     │ ammonia.land_cost                    
       labor_cost                    │    28,518,441.16 │ USD/year  │ 1     │ ammonia.labor_cost                   
       general_administration_cost   │     1,445,938.00 │ USD/year  │ 1     │ ammonia.general_administration_cost  
       property_tax_insurance        │    23,034,750.54 │ USD/year  │ 1     │ ammonia.property_tax_insurance       
       maintenance_cost              │     4,037,752.62 │ USD/year  │ 1     │ ammonia.maintenance_cost             
       cost_year                     │       2,022.0000 │ n/a       │ n/a   │ ammonia.cost_year                    
     synloop_ammonia_performance     │                  │           │       │                                      
       ammonia_out                   │      45,312.8682 │ kg/h      │ 8760  │ ammonia.ammonia_out                  
       nitrogen_out                  │       321,884.17 │ kg/h      │ 8760  │ ammonia.nitrogen_out                 
       hydrogen_out                  │       3,929.5472 │ kg/h      │ 8760  │ ammonia.hydrogen_out                 
       electricity_out               │       3,961.1244 │ MW        │ 8760  │ ammonia.electricity_out              
       heat_out                      │      37,609.4812 │ kW*h/kg   │ 8760  │ ammonia.heat_out                     
       catalyst_mass                 │       108,716.53 │ kg        │ 1     │ ammonia.catalyst_mass                
       total_ammonia_produced        │   396,940,725.31 │ kg/year   │ 1     │ ammonia.total_ammonia_produced       
       total_hydrogen_consumed       │    72,043,515.94 │ kg/year   │ 1     │ ammonia.total_hydrogen_consumed      
       total_nitrogen_consumed       │ 3,003,314,070.67 │ kg/year   │ 1     │ ammonia.total_nitrogen_consumed      
       total_electricity_consumed    │   245,334,157.62 │ kW*h/year │ 1     │ ammonia.total_electricity_consumed   
       limiting_input                │           2.4565 │ n/a       │ 8760  │ ammonia.limiting_input               
       max_hydrogen_capacity         │       1,000.0000 │ kg/h      │ 1     │ ammonia.max_hyd

### `resize_by_max_feedstock` mode

In this case, the electrolyzer will be sized to match the maximum `electricity_in` coming from HOPP
This increases the electrolyzer size to 1080 MW, the closest multiple of 40 MW (the cluster size) matching the max HOPP power output of 1048 MW.
This increases the LCOH to $4.98/kg H2, and increases the LCOA to $1.32/kg NH3, since electrolyzer is now oversized to utilize all of the HOPP electricity at peak output but thus has a lower hydrogen production capacity factor.

In [2]:
# Create a H2Integrate model
feed_model = H2IntegrateModel("14_size_mode_feedstock.yaml")

# Run the model
feed_model.run()

feed_model.post_process()

XDSM diagram written to connections_xdsm.pdf


77 explicit inputs:

╷                  ╷           ╷       ╷                                      
 Variable                            │             Mean │ Units     │ Shape │ Promoted name                        
╶────────────────────────────────────┼──────────────────┼───────────┼───────┼─────────────────────────────────────╴
 plant                               │                  │           │       │                                      
   ammonia                           │                  │           │       │                                      
     synloop_ammonia_cost            │                  │           │       │                                      
       total_ammonia_produced        │   397,680,920.91 │ kg/year   │ 1     │ ammonia.total_ammonia_produced       
       total_hydrogen_consumed       │    81,858,338.08 │ kg/year   │ 1     │ ammonia.total_hydrogen_consumed      
       total_nitrogen_consumed       │ 3,412,469,468.52 │ kg/year   │ 1     │ ammonia.total_nitrogen_consumed      
       total_electricity_consumed    │   245,334,157.62 │ kW*h/year │ 1     │ ammonia.total_electricity_consumed   
     synloop_ammonia_performance     │                  │           │       │                                      
       hydrogen_in                   │       9,344.5591 │ kg/h      │ 8760  │ ammonia.hydrogen_in                  
       nitrogen_in                   │           0.0000 │ kg/h      │ 8760  │ ammonia.nitrogen_in                  
       electricity_in                │           0.0000 │ MW        │ 8760  │ ammonia.electricity_in               
   electrolyzer                      │                  │           │       │                                      
     eco_pem_electrolyzer_performanc │                  │           │       │                                      
 e                                   │                  │           │       │                                      
       electricity_in                │       465,297.78 │ kW        │ 8760  │ electrolyzer.electricity_in          
     singlitico_electrolyzer_cost    │                  │           │       │                                      
       electricity_in                │       465,297.78 │ kW        │ 8760  │ electrolyzer.electricity_in          
       n_clusters                    │          16.0000 │ unitless  │ 1     │ electrolyzer.n_clusters              
       max_hydrogen_capacity         │       1,000.0000 │ kg/h      │ 1     │ electrolyzer.max_hydrogen_capacity   
       total_hydrogen_produced       │    78,497,673.82 │ kg/year   │ 1     │ electrolyzer.total_hydrogen_produced 
       electrolyzer_size_mw_cost     │       1,080.0000 │ MW        │ 1     │ electrolyzer.electrolyzer_size_mw_co 
                                     │                  │           │       │ st                                   
   electrolyzer_to_h2_storage_pipe   │                  │           │       │                                      
     hydrogen_in                     │           2.5957 │ kg/s      │ 8760  │ electrolyzer_to_h2_storage_pipe.hydr 
                                     │                  │           │       │ ogen_in                              
   finance_subgroup_h2               │                  │           │       │                                      
     adjusted_capex_opex_comp        │                  │           │       │                                      
       capex_hopp                    │ 1,754,699,616.00 │ USD       │ 1     │ finance_subgroup_h2.capex_hopp       
       opex_hopp                     │    32,953,490.40 │ USD/year  │ 1     │ finance_subgroup_h2.opex_hopp        
       varopex_hopp                  │           0.0000 │ USD/year  │ 30    │ finance_subgroup_h2.varopex_hopp     
       capex_electrolyzer            │ 1,013,196,133.76 │ USD       │ 1     │ finance_subgroup_h2.capex_electrolyz 
                                     │                  │           │       │ er             

107 explicit outputs:

╷                  ╷           ╷       ╷                                      
 Variable                            │             Mean │ Units     │ Shape │ Promoted name                        
╶────────────────────────────────────┼──────────────────┼───────────┼───────┼─────────────────────────────────────╴
 plant                               │                  │           │       │                                      
   ammonia                           │                  │           │       │                                      
     synloop_ammonia_cost            │                  │           │       │                                      
       CapEx                         │ 1,151,737,526.88 │ USD       │ 1     │ ammonia.CapEx                        
       OpEx                          │    25,480,837.34 │ USD/year  │ 1     │ ammonia.OpEx                         
       VarOpEx                       │           0.0000 │ USD/year  │ 30    │ ammonia.VarOpEx                      
       capex_asu                     │   347,075,632.70 │ USD       │ 1     │ ammonia.capex_asu                    
       capex_synloop                 │   443,088,488.84 │ USD       │ 1     │ ammonia.capex_synloop                
       capex_boiler                  │    10,355,840.22 │ USD       │ 1     │ ammonia.capex_boiler                 
       capex_cooling_tower           │     7,030,562.36 │ USD       │ 1     │ ammonia.capex_cooling_tower          
       capex_direct                  │   807,550,524.13 │ USD       │ 1     │ ammonia.capex_direct                 
       capex_depreciable_nonequipmen │   344,187,002.75 │ USD       │ 1     │ ammonia.capex_depreciable_nonequipme 
 t                                   │                  │           │       │ nt                                   
       land_cost                     │     5,015,782.62 │ USD       │ 1     │ ammonia.land_cost                    
       labor_cost                    │    28,518,441.16 │ USD/year  │ 1     │ ammonia.labor_cost                   
       general_administration_cost   │     1,445,938.00 │ USD/year  │ 1     │ ammonia.general_administration_cost  
       property_tax_insurance        │    23,034,750.54 │ USD/year  │ 1     │ ammonia.property_tax_insurance       
       maintenance_cost              │     4,037,752.62 │ USD/year  │ 1     │ ammonia.maintenance_cost             
       cost_year                     │       2,022.0000 │ n/a       │ n/a   │ ammonia.cost_year                    
     synloop_ammonia_performance     │                  │           │       │                                      
       ammonia_out                   │      45,397.3654 │ kg/h      │ 8760  │ ammonia.ammonia_out                  
       nitrogen_out                  │       368,552.32 │ kg/h      │ 8760  │ ammonia.nitrogen_out                 
       hydrogen_out                  │       5,041.9523 │ kg/h      │ 8760  │ ammonia.hydrogen_out                 
       electricity_out               │       3,916.2864 │ MW        │ 8760  │ ammonia.electricity_out              
       heat_out                      │      37,679.6135 │ kW*h/kg   │ 8760  │ ammonia.heat_out                     
       catalyst_mass                 │       108,919.26 │ kg        │ 1     │ ammonia.catalyst_mass                
       total_ammonia_produced        │   397,680,920.91 │ kg/year   │ 1     │ ammonia.total_ammonia_produced       
       total_hydrogen_consumed       │    81,858,338.08 │ kg/year   │ 1     │ ammonia.total_hydrogen_consumed      
       total_nitrogen_consumed       │ 3,412,469,468.52 │ kg/year   │ 1     │ ammonia.total_nitrogen_consumed      
       total_electricity_consumed    │   245,334,157.62 │ kW*h/year │ 1     │ ammonia.total_electricity_consumed   
       limiting_input                │           2.4688 │ n/a       │ 8760  │ ammonia.limiting_input               
       max_hydrogen_capacity         │       1,000.0000 │ kg/h      │ 1     │ ammonia.max_hyd

### `resize_by_max_product` mode: non-iterative

In this case, the electrolyzer will be sized to match the maximum `hydrogen_in` used by ammonia production
This decreases the electrolyzer size to 280 MW, the closest multiple of 40 MW (the cluster size) that will ensure an h2 produciton capacity that matches the ammonia plant's h2 intake at its max ammonia produciton capacity.
This increases the LCOH to $6.75/kg H2, but reduces the LCOA to $0.99/kg NH3, since electrolyzer size was matched to ammonia produciton but not HOPP.

In [3]:
# Create a H2Integrate model
prod_model = H2IntegrateModel("14_size_mode_commodity.yaml")

# Run the model
prod_model.run()

prod_model.post_process()

XDSM diagram written to connections_xdsm.pdf


77 explicit inputs:

╷                  ╷           ╷       ╷                                      
 Variable                            │             Mean │ Units     │ Shape │ Promoted name                        
╶────────────────────────────────────┼──────────────────┼───────────┼───────┼─────────────────────────────────────╴
 plant                               │                  │           │       │                                      
   ammonia                           │                  │           │       │                                      
     synloop_ammonia_cost            │                  │           │       │                                      
       total_ammonia_produced        │   394,127,860.43 │ kg/year   │ 1     │ ammonia.total_ammonia_produced       
       total_hydrogen_consumed       │    39,649,035.41 │ kg/year   │ 1     │ ammonia.total_hydrogen_consumed      
       total_nitrogen_consumed       │ 1,652,869,163.67 │ kg/year   │ 1     │ ammonia.total_nitrogen_consumed      
       total_electricity_consumed    │   245,334,157.62 │ kW*h/year │ 1     │ ammonia.total_electricity_consumed   
     synloop_ammonia_performance     │                  │           │       │                                      
       hydrogen_in                   │       4,526.1456 │ kg/h      │ 8760  │ ammonia.hydrogen_in                  
       nitrogen_in                   │           0.0000 │ kg/h      │ 8760  │ ammonia.nitrogen_in                  
       electricity_in                │           0.0000 │ MW        │ 8760  │ ammonia.electricity_in               
   electrolyzer                      │                  │           │       │                                      
     eco_pem_electrolyzer_performanc │                  │           │       │                                      
 e                                   │                  │           │       │                                      
       electricity_in                │       465,297.78 │ kW        │ 8760  │ electrolyzer.electricity_in          
     singlitico_electrolyzer_cost    │                  │           │       │                                      
       electricity_in                │       465,297.78 │ kW        │ 8760  │ electrolyzer.electricity_in          
       n_clusters                    │          16.0000 │ unitless  │ 1     │ electrolyzer.n_clusters              
       max_hydrogen_capacity         │       1,000.0000 │ kg/h      │ 1     │ electrolyzer.max_hydrogen_capacity   
       total_hydrogen_produced       │    37,993,410.83 │ kg/year   │ 1     │ electrolyzer.total_hydrogen_produced 
       electrolyzer_size_mw_cost     │         280.0000 │ MW        │ 1     │ electrolyzer.electrolyzer_size_mw_co 
                                     │                  │           │       │ st                                   
   electrolyzer_to_h2_storage_pipe   │                  │           │       │                                      
     hydrogen_in                     │           1.2573 │ kg/s      │ 8760  │ electrolyzer_to_h2_storage_pipe.hydr 
                                     │                  │           │       │ ogen_in                              
   finance_subgroup_h2               │                  │           │       │                                      
     adjusted_capex_opex_comp        │                  │           │       │                                      
       capex_hopp                    │ 1,754,699,616.00 │ USD       │ 1     │ finance_subgroup_h2.capex_hopp       
       opex_hopp                     │    32,953,490.40 │ USD/year  │ 1     │ finance_subgroup_h2.opex_hopp        
       varopex_hopp                  │           0.0000 │ USD/year  │ 30    │ finance_subgroup_h2.varopex_hopp     
       capex_electrolyzer            │   262,680,479.12 │ USD       │ 1     │ finance_subgroup_h2.capex_electrolyz 
                                     │                  │           │       │ er             

107 explicit outputs:

╷                  ╷           ╷       ╷                                      
 Variable                            │             Mean │ Units     │ Shape │ Promoted name                        
╶────────────────────────────────────┼──────────────────┼───────────┼───────┼─────────────────────────────────────╴
 plant                               │                  │           │       │                                      
   ammonia                           │                  │           │       │                                      
     synloop_ammonia_cost            │                  │           │       │                                      
       CapEx                         │ 1,151,737,526.88 │ USD       │ 1     │ ammonia.CapEx                        
       OpEx                          │    25,507,976.66 │ USD/year  │ 1     │ ammonia.OpEx                         
       VarOpEx                       │           0.0000 │ USD/year  │ 30    │ ammonia.VarOpEx                      
       capex_asu                     │   347,075,632.70 │ USD       │ 1     │ ammonia.capex_asu                    
       capex_synloop                 │   443,088,488.84 │ USD       │ 1     │ ammonia.capex_synloop                
       capex_boiler                  │    10,355,840.22 │ USD       │ 1     │ ammonia.capex_boiler                 
       capex_cooling_tower           │     7,030,562.36 │ USD       │ 1     │ ammonia.capex_cooling_tower          
       capex_direct                  │   807,550,524.13 │ USD       │ 1     │ ammonia.capex_direct                 
       capex_depreciable_nonequipmen │   344,187,002.75 │ USD       │ 1     │ ammonia.capex_depreciable_nonequipme 
 t                                   │                  │           │       │ nt                                   
       land_cost                     │     5,015,782.62 │ USD       │ 1     │ ammonia.land_cost                    
       labor_cost                    │    28,518,441.16 │ USD/year  │ 1     │ ammonia.labor_cost                   
       general_administration_cost   │     1,445,938.00 │ USD/year  │ 1     │ ammonia.general_administration_cost  
       property_tax_insurance        │    23,034,750.54 │ USD/year  │ 1     │ ammonia.property_tax_insurance       
       maintenance_cost              │     4,037,752.62 │ USD/year  │ 1     │ ammonia.maintenance_cost             
       cost_year                     │       2,022.0000 │ n/a       │ n/a   │ ammonia.cost_year                    
     synloop_ammonia_performance     │                  │           │       │                                      
       ammonia_out                   │      44,991.7649 │ kg/h      │ 8760  │ ammonia.ammonia_out                  
       nitrogen_out                  │       167,872.32 │ kg/h      │ 8760  │ ammonia.nitrogen_out                 
       hydrogen_out                  │         261.9802 │ kg/h      │ 8760  │ ammonia.hydrogen_out                 
       electricity_out               │       4,131.5164 │ MW        │ 8760  │ ammonia.electricity_out              
       heat_out                      │      37,342.9669 │ kW*h/kg   │ 8760  │ ammonia.heat_out                     
       catalyst_mass                 │       107,946.13 │ kg        │ 1     │ ammonia.catalyst_mass                
       total_ammonia_produced        │   394,127,860.43 │ kg/year   │ 1     │ ammonia.total_ammonia_produced       
       total_hydrogen_consumed       │    39,649,035.41 │ kg/year   │ 1     │ ammonia.total_hydrogen_consumed      
       total_nitrogen_consumed       │ 1,652,869,163.67 │ kg/year   │ 1     │ ammonia.total_nitrogen_consumed      
       total_electricity_consumed    │   245,334,157.62 │ kW*h/year │ 1     │ ammonia.total_electricity_consumed   
       limiting_input                │           1.8903 │ n/a       │ 8760  │ ammonia.limiting_input               
       max_hydrogen_capacity         │       1,000.0000 │ kg/h      │ 1     │ ammonia.max_hyd

### `resize_by_max_product` mode: iterative

In this case, the electrolyzer will be sized to match the `maximum_hydrogen_capacity` connected variable from `ammonia`
This creates a feedback loop in the larger OpenMDAO problem that must be solved iteratively.
Unfortunately, ProFast does not seem to be able to handle this yet and is crashing at `pf.solve_price()`

In [4]:
# Create a H2Integrate model
prod_model = H2IntegrateModel("14_size_mode_commodity_iterative.yaml")

# Run the model
prod_model.run()

prod_model.post_process()

XDSM diagram written to connections_xdsm.pdf

=====
plant
=====
NL: NLBGS Converged in 5 iterations


77 explicit inputs:

╷                  ╷           ╷       ╷                                      
 Variable                            │             Mean │ Units     │ Shape │ Promoted name                        
╶────────────────────────────────────┼──────────────────┼───────────┼───────┼─────────────────────────────────────╴
 plant                               │                  │           │       │                                      
   ammonia                           │                  │           │       │                                      
     synloop_ammonia_cost            │                  │           │       │                                      
       total_ammonia_produced        │   394,127,860.43 │ kg/year   │ 1     │ ammonia.total_ammonia_produced       
       total_hydrogen_consumed       │    39,649,035.41 │ kg/year   │ 1     │ ammonia.total_hydrogen_consumed      
       total_nitrogen_consumed       │ 1,652,869,163.67 │ kg/year   │ 1     │ ammonia.total_nitrogen_consumed      
       total_electricity_consumed    │   245,334,157.62 │ kW*h/year │ 1     │ ammonia.total_electricity_consumed   
     synloop_ammonia_performance     │                  │           │       │                                      
       hydrogen_in                   │       4,526.1456 │ kg/h      │ 8760  │ ammonia.hydrogen_in                  
       nitrogen_in                   │           0.0000 │ kg/h      │ 8760  │ ammonia.nitrogen_in                  
       electricity_in                │           0.0000 │ MW        │ 8760  │ ammonia.electricity_in               
   electrolyzer                      │                  │           │       │                                      
     eco_pem_electrolyzer_performanc │                  │           │       │                                      
 e                                   │                  │           │       │                                      
       electricity_in                │       465,297.78 │ kW        │ 8760  │ electrolyzer.electricity_in          
     singlitico_electrolyzer_cost    │                  │           │       │                                      
       electricity_in                │       465,297.78 │ kW        │ 8760  │ electrolyzer.electricity_in          
       n_clusters                    │          16.0000 │ unitless  │ 1     │ electrolyzer.n_clusters              
       max_hydrogen_capacity         │       5,294.6801 │ kg/h      │ 1     │ electrolyzer.max_hydrogen_capacity   
       total_hydrogen_produced       │    37,993,410.83 │ kg/year   │ 1     │ electrolyzer.total_hydrogen_produced 
       electrolyzer_size_mw_cost     │         280.0000 │ MW        │ 1     │ electrolyzer.electrolyzer_size_mw_co 
                                     │                  │           │       │ st                                   
   electrolyzer_to_h2_storage_pipe   │                  │           │       │                                      
     hydrogen_in                     │           1.2573 │ kg/s      │ 8760  │ electrolyzer_to_h2_storage_pipe.hydr 
                                     │                  │           │       │ ogen_in                              
   finance_subgroup_h2               │                  │           │       │                                      
     adjusted_capex_opex_comp        │                  │           │       │                                      
       capex_hopp                    │ 1,754,699,616.00 │ USD       │ 1     │ finance_subgroup_h2.capex_hopp       
       opex_hopp                     │    32,953,490.40 │ USD/year  │ 1     │ finance_subgroup_h2.opex_hopp        
       varopex_hopp                  │           0.0000 │ USD/year  │ 30    │ finance_subgroup_h2.varopex_hopp     
       capex_electrolyzer            │   262,680,479.12 │ USD       │ 1     │ finance_subgroup_h2.capex_electrolyz 
                                     │                  │           │       │ er             

107 explicit outputs:

╷                  ╷           ╷       ╷                                      
 Variable                            │             Mean │ Units     │ Shape │ Promoted name                        
╶────────────────────────────────────┼──────────────────┼───────────┼───────┼─────────────────────────────────────╴
 plant                               │                  │           │       │                                      
   ammonia                           │                  │           │       │                                      
     synloop_ammonia_cost            │                  │           │       │                                      
       CapEx                         │ 1,151,737,526.88 │ USD       │ 1     │ ammonia.CapEx                        
       OpEx                          │    25,507,976.66 │ USD/year  │ 1     │ ammonia.OpEx                         
       VarOpEx                       │           0.0000 │ USD/year  │ 30    │ ammonia.VarOpEx                      
       capex_asu                     │   347,075,632.70 │ USD       │ 1     │ ammonia.capex_asu                    
       capex_synloop                 │   443,088,488.84 │ USD       │ 1     │ ammonia.capex_synloop                
       capex_boiler                  │    10,355,840.22 │ USD       │ 1     │ ammonia.capex_boiler                 
       capex_cooling_tower           │     7,030,562.36 │ USD       │ 1     │ ammonia.capex_cooling_tower          
       capex_direct                  │   807,550,524.13 │ USD       │ 1     │ ammonia.capex_direct                 
       capex_depreciable_nonequipmen │   344,187,002.75 │ USD       │ 1     │ ammonia.capex_depreciable_nonequipme 
 t                                   │                  │           │       │ nt                                   
       land_cost                     │     5,015,782.62 │ USD       │ 1     │ ammonia.land_cost                    
       labor_cost                    │    28,518,441.16 │ USD/year  │ 1     │ ammonia.labor_cost                   
       general_administration_cost   │     1,445,938.00 │ USD/year  │ 1     │ ammonia.general_administration_cost  
       property_tax_insurance        │    23,034,750.54 │ USD/year  │ 1     │ ammonia.property_tax_insurance       
       maintenance_cost              │     4,037,752.62 │ USD/year  │ 1     │ ammonia.maintenance_cost             
       cost_year                     │       2,022.0000 │ n/a       │ n/a   │ ammonia.cost_year                    
     synloop_ammonia_performance     │                  │           │       │                                      
       ammonia_out                   │      44,991.7649 │ kg/h      │ 8760  │ ammonia.ammonia_out                  
       nitrogen_out                  │       167,872.32 │ kg/h      │ 8760  │ ammonia.nitrogen_out                 
       hydrogen_out                  │         261.9802 │ kg/h      │ 8760  │ ammonia.hydrogen_out                 
       electricity_out               │       4,131.5164 │ MW        │ 8760  │ ammonia.electricity_out              
       heat_out                      │      37,342.9669 │ kW*h/kg   │ 8760  │ ammonia.heat_out                     
       catalyst_mass                 │       107,946.13 │ kg        │ 1     │ ammonia.catalyst_mass                
       total_ammonia_produced        │   394,127,860.43 │ kg/year   │ 1     │ ammonia.total_ammonia_produced       
       total_hydrogen_consumed       │    39,649,035.41 │ kg/year   │ 1     │ ammonia.total_hydrogen_consumed      
       total_nitrogen_consumed       │ 1,652,869,163.67 │ kg/year   │ 1     │ ammonia.total_nitrogen_consumed      
       total_electricity_consumed    │   245,334,157.62 │ kW*h/year │ 1     │ ammonia.total_electricity_consumed   
       limiting_input                │           1.8903 │ n/a       │ 8760  │ ammonia.limiting_input               
       max_hydrogen_capacity         │       5,294.6801 │ kg/h      │ 1     │ ammonia.max_hyd